In [ ]:
import matplotlib.pyplot as plt
import numpy             as np
import pandas            as pd
import seaborn           as sns
import libraries.model   as clm
import libraries.dataset as cld
import os
import torch
import json
import sys

from pymatgen.core.structure         import Structure
from pymatgen.symmetry.bandstructure import HighSymmKpath

sys.path.append('../../UPC')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sns.set_theme()

In [ ]:
model_folder    = 'model'  # Pre-trained model and dataset parameters
output_folder   = 'output'  # Output files and figures
input_folder    = 'input'  # General files (e.g., atomic masses information)

# Whether to plot the harmonic extrapolations of Fv (very time-consuming) or not
plot_extrapolations = False

# Defining the range of temperatures
Ti = 300
Tf = 600
dT = 50
temperatures = np.arange(Ti, Tf+dT, dT)  # Temperatures for prediction of free-energies

# Loading dictionary of atomic masses
atomic_masses = cld.load_atomic_masses(f'{input_folder}/atomic_masses.dat')

In [ ]:
# Load the data from the JSON file
with open(f'{model_folder}/standardized_parameters.json', 'r') as json_file:
    numpy_dict = json.load(json_file)

# Convert NumPy arrays back to PyTorch tensors
standardized_parameters = {}
for key, value in numpy_dict.items():
    try:
        standardized_parameters[key] = torch.tensor(value)
    except:
        standardized_parameters[key] = value

# Load reference dataset for uncertainty estimation
reference_dataset = torch.load(f'{model_folder}/ref_dataset_std.pt', weights_only=False)

# Load Graph Neural Network model (making room for temperature as node attribute) to device
# Dropout for initializing the model, not used at all while predicting
model = clm.GCNN(features_channels=5,
                 pdropout=0).to(device)

# Load and evaluate Graph Neural Network model
model.load_state_dict(torch.load(f'{model_folder}/model.pt', map_location=torch.device(device)))
model.eval()

In [ ]:
uncertainty_data = cld.load_json(f'{model_folder}/uncertainty_data.json')
    
interpolator = clm.fit_interpolator(uncertainty_data['uncertainty_values'], reference_dataset, model)
del reference_dataset

In [ ]:
def pred_ML_Fv(target_database, temperatures, standardized_parameters, model_folder, device):
    # Create dataset for predictions
    dataset = cld.create_predictions_dataset(target_database, path_to_material=True, path_to_polymorph=True)
    
    labels = [graph.label for graph in dataset]
    
    # Standardize properties
    std_dataset = cld.standardize_dataset_from_keys(dataset, standardized_parameters)
    del dataset
    
    # Include temperatures
    std_dataset_w_temp = cld.include_temperatures(std_dataset, temperatures, standardized_parameters)
    
    # Free-up CUDA
    del std_dataset
    
    # Compute predictions and corresponding uncertainties
    predictions, uncertainties = clm.make_predictions(std_dataset_w_temp,  model, standardized_parameters,
                                                      uncertainty_data, interpolator)
    
    # Free-up CUDA
    del std_dataset_w_temp
    
    # Computing the coefficients and uncertainties from fitting
    coefficients = clm.compute_coefficients(temperatures, predictions)
    
    # Compute Fv
    Fv_pred = clm.compute_Fv(temperatures, coefficients)[0]
    return  Fv_pred, uncertainties

def pred_DFT_Fv(path_to_PHONON, temperatures):
    # Loading number of atoms
    _, _, concentration, _ = MPL.information_from_VASPfile(path_to_PHONON, file='POSCAR')
    n_atoms = np.sum(concentration)
    
    # Reading supercell information
    dim_info = MPL.read_phonopyconf(path_to_PHONON)
    
    # Write mesh.conf file (needed for phonopy)
    MPL.write_meshconf(path_to_PHONON, material, dim_info, Ti, Tf, dT)
    
    # Getting thermal properties with phonopy (ignoring output)
    previous_dir = os.getcwd()
    os.chdir(path_to_PHONON)
    os.system('phonopy -t mesh.conf > /dev/null')
    os.chdir(previous_dir)
    
    # Read generated thermal properties (kJ/mol)
    _, Fv_PHONON = MPL.read_thermalpropertyyaml(len(temperatures), path_to_PHONON, thermalproperty='free_energy')
    
    # Pass kJ / molmp-1009220 to meV / atom
    conversion_factor = 1.6 * 6.022 * 0.01 * n_atoms
    Fv_PHONON        /= conversion_factor
    return Fv_PHONON

In [ ]:
materials = {
    'MgS-reversed': {
        'F-43m',
        'Fm-3m',
        'P6_3-mmc'
    },
    'MgS': {
        'F-43m',
        'Fm-3m',
        'P6_3-mmc'
    },
    'MgF2': {
        'P-3m1',
        'Pnnm'
    },
    'NaH': {
        'Fm-3m',
        'Pm-3m'
    },
    'NaH-reversed': {
        'Fm-3m',
        'Pm-3m'
    }
}

In [ ]:
colors = ['b', 'r', 'g', 'k']
for material in materials.keys():
    print(material)
    for i, polymorph in enumerate(materials[material]):
        path_to_phonons = f'/home/claudio/Desktop/cibran-varios/Fv-things/validation-phonons/{material}/{polymorph}'
        
        Fv_ML, uncert_ML = pred_ML_Fv(path_to_phonons,
                                      temperatures, standardized_parameters, model_folder, device)
        Fv_DFT = pred_DFT_Fv(path_to_phonons,
                             temperatures)

        uncert_ML = np.ones_like(uncert_ML) * np.max(np.abs(uncert_ML))
        
        # Plotting
        epa = np.loadtxt(f'{path_to_phonons}/EPA') * 1e3
        if polymorph == 'Fm-3m':
            label = r'Fm$\overline{3}$m'
        elif polymorph == 'P6_3-mmc':
            label = r'P6$_3$/mmc'
        elif polymorph == 'F-43m':
            label = r'F$\overline{4}$3m'
        elif polymorph == 'P6_3-mmc':
            label = r'P6$_3$/mmc'
        elif polymorph == 'P-3m1':
            label = r'P$\overline{3}$m1'
        elif polymorph == 'Pnnm':
            label = r'Pnnm'
        elif polymorph == 'Pm-3m':
            label = r'Pm$\overline{3}$m'
        
        plt.errorbar(temperatures, epa+Fv_ML, yerr=uncert_ML, fmt='o', color=colors[i])
        plt.plot(temperatures,     epa+Fv_DFT, label=label,           color=colors[i])
            
    plt.xlabel(r'$T$ (K)')
    plt.ylabel(r'$\Delta F$ (meV/atom)')
    plt.legend(loc='best')
    plt.savefig(f'{material}.pdf', dpi=50, bbox_inches='tight')
    plt.show()

In [ ]:
colors = ['b', 'r', 'g', 'k']

materials = {
    'CO': ['P2_12_12_1', 'R3c'],
    'MgTe': ['F-43m', 'P6_3mc'],
    'SeN': ['C2-c', 'P2_1-c'],
    'SrC2': ['C2-c', 'I4-mmm'],
    'WCl6': ['P-3m1', 'R-3'],
    'WN2': ['P3_121', 'Pna2_1'],
    'WSe2': ['P6_3-mmc', 'P-6m2'],
    'ZnCl2': ['P2_1-c', 'P4_2-nmc'],
    'ZnTe': ['F-43m', 'P6_3mc'],
    'ZrSeO': ['P2_13', 'P4-nmm']
}

In [ ]:
uncert_DFT_0 = np.ones(len(temperatures)) * 4
uncert_DFT_1 = np.ones(len(temperatures)) * 4

uplims = [False] * len(temperatures)
lolims = [False] * len(temperatures)

for material in materials.keys():
    print(material)

    polymorphs = materials[material]
    path_to_material = f'/home/claudio/Desktop/validation/{material}'

    # IDX = 0
    idx = 0
    path_to_polymorph = f'{path_to_material}/{polymorphs[idx]}'
    path_to_MP = f'{path_to_polymorph}/relaxation'
    path_to_phonons = f'{path_to_polymorph}/phonons'

    if not os.path.exists(path_to_MP):
        continue
    
    Fv_ML_MP_0, uncert_ML_MP_0 = pred_ML_Fv(path_to_MP,
                                            temperatures, standardized_parameters, model_folder, device)
    Fv_ML_phonons_0, uncert_ML_phonons_0 = pred_ML_Fv(path_to_phonons,
                                                      temperatures, standardized_parameters, model_folder, device)
    Fv_DFT_0 = pred_DFT_Fv(path_to_phonons,
                           temperatures)

    uncert_ML_MP_0 = np.ones_like(uncert_ML_MP_0) * np.max(np.abs(uncert_ML_MP_0))
    uncert_ML_phonons_0 = np.ones_like(uncert_ML_phonons_0) * np.max(np.abs(uncert_ML_phonons_0))
    epa_0 = float(np.loadtxt(f'{path_to_MP}/EPA')) * 1e3  # From eV/atom to meV/atom

    
    # IDX = 1
    idx = 1
    path_to_polymorph = f'{path_to_material}/{polymorphs[idx]}'
    path_to_MP = f'{path_to_polymorph}/relaxation'
    path_to_phonons = f'{path_to_polymorph}/phonons'

    if not os.path.exists(path_to_MP):
        continue
    
    Fv_ML_MP_1, uncert_ML_MP_1 = pred_ML_Fv(path_to_MP,
                                            temperatures, standardized_parameters, model_folder, device)
    Fv_ML_phonons_1, uncert_ML_phonons_1 = pred_ML_Fv(path_to_phonons,
                                                      temperatures, standardized_parameters, model_folder, device)
    Fv_DFT_1 = pred_DFT_Fv(path_to_phonons,
                           temperatures)

    uncert_ML_MP_1 = np.ones_like(uncert_ML_MP_1) * np.max(np.abs(uncert_ML_MP_1))
    uncert_ML_phonons_1 = np.ones_like(uncert_ML_phonons_1) * np.max(np.abs(uncert_ML_phonons_1))
    epa_1 = float(np.loadtxt(f'{path_to_MP}/EPA')) * 1e3  # From eV/atom to meV/atom

    
    # Diff
    F_ML_MP_diff = (epa_1+Fv_ML_MP_1) - (epa_0+Fv_ML_MP_0)
    uncert_ML_MP_diff = np.sqrt(uncert_ML_MP_0**2 + uncert_ML_MP_1**2)

    F_ML_phonons_diff = (epa_1+Fv_ML_phonons_1) - (epa_0+Fv_ML_phonons_0)
    uncert_ML_phonons_diff = np.sqrt(uncert_ML_phonons_0**2 + uncert_ML_phonons_1**2)

    F_DFT_diff = (epa_1+Fv_DFT_1) - (epa_0+Fv_DFT_0)
    uncert_DFT_diff = np.sqrt(uncert_DFT_0**2 + uncert_DFT_1**2)
    
    # Plotting
    label = f'{polymorphs[1]}-{polymorphs[0]}'
    plt.errorbar(temperatures, F_ML_MP_diff,      color=colors[0], yerr=uncert_ML_MP_diff,      label='ML original', 
                 fmt=':o', uplims=uplims, lolims=lolims)
    plt.errorbar(temperatures, F_ML_phonons_diff, color=colors[1], yerr=uncert_ML_phonons_diff, label='ML relaxed',
                 fmt=':o', uplims=uplims, lolims=lolims)
    plt.errorbar(temperatures, F_DFT_diff,        color=colors[2], yerr=uncert_DFT_diff,        label='DFT',
                 fmt=':o', uplims=uplims, lolims=lolims)
        
    plt.xlabel(r'$T$ (K)')
    plt.ylabel(r'$\Delta F$ (meV/atom)')
    plt.legend(loc='best')
    plt.savefig(f'{material}-diff.pdf', dpi=50, bbox_inches='tight')
    plt.show()